In [ ]:
!pip freeze > requirements.txt

### Setup

In [ ]:
import sys

if 'google.colab' in sys.modules:
    %pip install -q -U keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.5 MB/s eta 0:00:00


In [ ]:
%pip install 'git+https://github.com/jeslago/epftoolbox.git'

  Cloning https://github.com/jeslago/epftoolbox.git to /tmp/pip-req-build-flwcfhbh
  Running command git clone --filter=blob:none --quiet https://github.com/jeslago/epftoolbox.git /tmp/pip-req-build-flwcfhbh
  Resolved https://github.com/jeslago/epftoolbox.git to commit 47d6e0629f65ebd19d3c12cb5689dbad0c2ea078
  Preparing metadata (setup.py) ... done
  Created wheel for epftoolbox: filename=epftoolbox-1.0-py3-none-any.whl size=58746 sha256=0aacbd85a17da3c91af58fed3efe4106d9790e8413fee45ccfe97fe852843762
  Stored in directory: /tmp/pip-ephem-wheel-cache-d12lta9c/wheels/f1/d7/c7/feb15c8f047f5b069656a52f3bcde89a7310a311f9cfac292a
Successfully built epftoolbox


In [ ]:
%pip install finta

In [ ]:
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 21.8 MB/s eta 0:00:00


In [ ]:
import os
import sys
from pathlib import Path

import finta
import keras_tuner as kt
import optuna
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn
import sklearn
import statsmodels
import tensorflow as tf
import tqdm
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

DRIVE_ROOT = Path('/content/drive/MyDrive')

PROJECT_ROOT = DRIVE_ROOT / 'projects/btc'
PROJECT_DATA_DIR = PROJECT_ROOT / 'data'
PROJECT_CLEANED_DIR = PROJECT_DATA_DIR / 'cleaned'
PROJECT_EXTERNAL_DIR = PROJECT_DATA_DIR / 'external'

CONFIG_FILENAME = 'config.yaml'
BTC_CLEANED_CSV_FILENAME = 'ohlcv_274_cleaned.csv'
FEDFUNDRATE_CSV_FILENAME = 'fedfunds.csv'
M2SL_CSV_FILENAME = 'm2sl.csv'

CONFIG_FILE_PATH = PROJECT_ROOT / CONFIG_FILENAME
BTC_CLEANED_FILE_PATH = PROJECT_CLEANED_DIR / BTC_CLEANED_CSV_FILENAME
FEDFUNDRATE_CSV_FILE_PATH = PROJECT_EXTERNAL_DIR / FEDFUNDRATE_CSV_FILENAME
M2SL_CSV_FILE_PATH = PROJECT_EXTERNAL_DIR / M2SL_CSV_FILENAME

Mounted at /content/drive


In [ ]:
project_root = os.path.abspath(PROJECT_ROOT)

if project_root not in sys.path:
  sys.path.append(project_root)

print(f'Project root \'{project_root}\' added to sys.path.')

Project root '/content/drive/MyDrive/projects/btc' added to sys.path.


In [ ]:
from src.analysis import *
from src.data_loader import *
from src.data_splitter import *
from src.data_cleaner import *
from src.model import *
from src.preprocessing import *
from src.utils import *
from src.visualization import *

In [ ]:
print('--- Python Interpreter and Standard Library Versions ---')
print(f'- python: {sys.version}')
print(f'- python info: {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}')

print(f'- os, hashlib, pathlib module: (version tied to Python interpreter)')

print('\n--- Third-Party Library Versions ---')
print(f'- keras_tuner: {kt.__version__}')
print(f'- optuna: {optuna.__version__}')
print(f'- matplotlib: {matplotlib.__version__}')
print(f'- numpy: {np.__version__}')
print(f'- pandas: {pd.__version__}')
print(f'- seaborn: {seaborn.__version__}')
print(f'- sklearn: {sklearn.__version__}')
print(f'- statsmodels: {statsmodels.__version__}')
print(f'- tensorflow: {tf.__version__}')
#print(f'- tqdm: {tqdm.__version__}')

print('\n--- Google Colab Specific Module Versions ---')
print(f'- google.colab.drive: (version tied to Colab environment)')

--- Python Interpreter and Standard Library Versions ---
- python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
- python info: 3.12.12
- os, hashlib, pathlib module: (version tied to Python interpreter)

--- Third-Party Library Versions ---
- keras_tuner: 1.4.7
- optuna: 4.5.0
- matplotlib: 3.10.0
- numpy: 2.0.2
- pandas: 2.2.2
- seaborn: 0.13.2
- sklearn: 1.6.1
- statsmodels: 0.14.5
- tensorflow: 2.19.0

--- Google Colab Specific Module Versions ---
- google.colab.drive: (version tied to Colab environment)


In [ ]:
%pip show epftoolbox

Name: epftoolbox
Version: 1.0
Summary: An open-access benchmark and toolbox for electricity price forecasting
Home-page: https://github.com/jeslago/epftoolbox
Author: Jesus Lago
Author-email: jesuslagogarcia@gmail.com
License: GNU AGPLv3
Location: /usr/local/lib/python3.12/dist-packages
Requires: hyperopt, keras, matplotlib, numpy, pandas, scikit-learn, scipy, statsmodels, tensorflow
Required-by: 


In [ ]:
%pip show finta

Name: finta
Version: 1.3
Summary: Common financial technical indicators implemented in Pandas.
Home-page: https://github.com/peerchemist/finta
Author: Peerchemist
Author-email: peerchemist@protonmail.ch
License: LGPLv3+
Location: /usr/local/lib/python3.12/dist-packages
Requires: numpy, pandas
Required-by: 


### Load the cleaned dataset.

In [ ]:
cleaned_ds = load_btc_ds(BTC_CLEANED_FILE_PATH, 'date', True)


Loading dataset from: ohlcv_274_cleaned.csv...
Dataset loaded successfully.


### Perform linear interpolation to fill the missing values on March 15, 2025 to ensure a continuous time series for GRU models.

In [ ]:
imputed_ds = fill_time_series_gaps(cleaned_ds, 'min')

### Resample the cleaned dataset into multiple timeframes.

In [ ]:
resampled_ds = resample_btc_data(imputed_ds)


--- Resampling BTC Data to Multiple Timeframes ---
- Resampling to hourly frequency...
- Resampling to daily frequency...
- Resampling to weekly frequency...
- Resampling to monthly frequency...
--- BTC Data Resampling Complete ---


### Split the resampled datasets into training, validation, and test sets of the target timeframe.

In [ ]:
train_ds, valid_ds, test_ds = split_btc_ds(resampled_ds['daily'], 'Day')


--- Splitting the BTC Dataset into Training, Validation, and Test Sets (Day) ---
--- BTC Dataset Splitting Complete ---


### Run the ADF test on datasets at the target frequency to verify stationarity and seasonality.

In [ ]:
run_adf_test(calculate_log_and_diff(train_ds.copy(), 'close'), 'log_returns_close', 'BTC Daily Closing Returns')

= ADF Test: BTC Daily Closing Returns =
ADF Statistics: -10.8445
p-value: 0.0000
Critical Values:
	1%:-3.4320
	5%:-2.8623
	10%:-2.5672
Conclusion: The p-value is less than or equal to 0.05. The data is likely stationary and seasonal.



### Prepare the dataset by consolidating data splits, adding a comprehensive suite of features, and then splitting the data back.

In [ ]:
train_prep_ds, valid_prep_ds, test_prep_ds = prepare_feature_ds(
    train_ds, valid_ds, test_ds,
    ['open', 'high', 'low', 'close', 'volume'],
    FEDFUNDRATE_CSV_FILE_PATH, M2SL_CSV_FILE_PATH
)

= Feature Engineering and Dataset Preparation =
Step 1: Concatenating data splits...
- Combined train, validation, and test sets into a single DataFrame. Full shape: (4922, 5)

Step 2: Engineering new features...
- Calculated log returns for ['open', 'high', 'low', 'close', 'volume'].
- Calculated the high-low price range as a percentage of the open price.
- Calculated Simple Moving Average (SMA) for trend.
- Calculated Relative Strength Index (RSI) for momentum.
- Calculated Average True Range (ATR) for volatility.
- Calculated On-Balance Volume (OBV) for volume pressure.
- Calculated Moving Average Convergence Divergence (MACD).
- Engineered relationship features
- Calculated technical indicators.


/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


- Macro data loaded and prepared.
- Macro data merged.
- Added macroeconomic features.

Step 3: Handling missing and infinite values...
- Replaced 0 infinite values (inf, -inf) with NaN.
- Forward-filled missing values to maintain data continuity.
- Shape before dropping NaNs: (4922, 28)
- Shape after dropping NaNs: (4873, 28)
- Dropped initial rows containing NaNs that resulted from feature calculations.

Step 4: Splitting into final datasets...
- Split the single processed DataFrame back into train, validation, and test sets.
- Train shape: (3888, 28)
- Valid shape: (492, 28)
- Test shape: (493, 28)
= Feature Preparation Complete =


### Fine-tune hyperparameters and utilize techniques.
1.   Architectural Hyperparameters
  *   Number of hidden layers (Conv1D and RNN)
      * Deeper networks often have higher parameter efficiency, meaning they can represent complex functions with fewer total parameters than wide, shallow networks.
  *   Number of neurons per RNN layer
      * As a general heuristic, increasing the number of layers (making the network deeper) is often more effective for improving performance than increasing the number of units in a single layer (making it wider).
  *   Number of filters per Conv1D layer
  *   Kernel size (Conv1D)
  *   Stride (Conv1D)
  *   Padding (Conv1D)
  *   Type of RNN cell (e.g., LSTM/GRU)
  *   Activation function (Conv1D, RNN and Dense)
      * A poor choice of activation function can lead to unstable gradients (e.g., vanishing or exploding gradients).

2.   Training and Optimization Hyperparameters
  *   Optimizer (e.g., Adam, Nadam)
      * The choice of optimizer can significantly impact convergence speed. Adaptive optimizers like Adam often converge faster than standard stochastic gradient descent.
  *   Learning rate
      * Often considered the most important hyperparameter. A common rule of thumb is that the optimal learning rate is roughly half the maximum value at which the training algorithm still converges.
  *   Batch size
      * This has a significant impact on both the model's final performance (generalization) and training time. Larger batches can speed up training but may lead to sharp minima, while smaller batches can offer a regularizing effect.

3.   Optimization and Regularization Techniques
  *   Kernel Initialization (e.g., Glorot/He initialization)
      * Proper initialization is crucial for preventing the initial gradients from being too large or too small.
  *   Normalization (Layer normalization)
      * Batch Normalization is difficult to use effectively with RNNs due to the varying sequence lengths.
      * Layer Normalization is a common alternative for RNNs that helps stabilize hidden state dynamics and prevent unstable gradients.
  *   Learning Rate Scheduling (e.g., ReduceLROnPlateau)
      * This involves dynamically adjusting the learning rate during training to improve convergence.
  *   Gradient Clipping
      * This technique directly mitigates the exploding gradients problem by capping the gradient's norm during backpropagation.
  *   L1/L2 Regularization
      * These methods add a penalty to the loss function based on the magnitude of the model's weights to discourage overly complex models.
  *   Dropout (Recurrent Dropout for RNN)
      * This randomly sets a fraction of neuron activations to zero during training to prevent co-adaptation between neurons.
  *   Early Stopping
      * This technique monitors a validation metric (e.g., validation loss) and stops training when the metric stops improving, which helps prevent overfitting and saves computational resources.

### GRU Model Tuning (1-Week)

In [ ]:
tf.random.set_seed(42)

In [ ]:
FULL_COLS = list(train_prep_ds.columns)
TARGET_COL = 'close'
INPUT_WINDOW = 7
TARGET_WINDOW = 7
FORECAST_HORIZON = 7

In [ ]:
naive_metrics, naive_pred = train_and_forecast_with_naive_model(valid_prep_ds, 'close', 7, 'Validation', 'Days')

= Naive Model Evaluation on Validation Set (Horizon: 7 Days) =
--- Fitting Naive Model ---

--- Evaluating Naive Model ---
- Mean Absolute Percentage Error (MAPE): 4.7134%
- Directional Accuracy (DA): 44.7699%

--- Generating Final Naive Forecast (Threshold: 0.5%) ---
- Forecast for 2024-02-22: $52003.00
- Directional Signal for 2024-02-22: 1.0


In [ ]:
gru_hps_and_metrics = find_best_rnn_hps(
    'gru', train_prep_ds, valid_prep_ds,
     FULL_COLS, TARGET_COL, FORECAST_HORIZON,
     INPUT_WINDOW, TARGET_WINDOW,
     'Days', naive_metrics, 15
)

Trial 15 Complete [00h 00m 35s]
val_mae: 0.3619118332862854

Best val_mae So Far: 0.36063024401664734
Total elapsed time: 00h 15m 47s

GRU hyperparameter tuning complete:
- n_conv_layers: 2
- kernel_size: 5
- n_rnn_layers: 3
- use_l2: True
- dropout: 0.30000000000000004
- learning_rate: 0.005117045969850107
- optimizer: adam
- clipnorm: 1.1
- filters_0: 96
- units_0: 96
- filters_1: 192
- units_1: 160
- filters_2: 224
- units_2: 32
- l2_rate: 0.005343634220268847

--- Fitting GRU Model ---
Epoch 1/100
121/121 - 15s - 121ms/step - loss: 1.1551 - mae: 0.7467 - val_loss: 0.1598 - val_mae: 0.3681 - learning_rate: 0.0051
Epoch 2/100
121/121 - 6s - 46ms/step - loss: 0.3324 - mae: 0.6185 - val_loss: 0.1405 - val_mae: 0.3635 - learning_rate: 0.0051
Epoch 3/100
121/121 - 5s - 45ms/step - loss: 0.3255 - mae: 0.6164 - val_loss: 0.1387 - val_mae: 0.3636 - learning_rate: 0.0051
Epoch 4/100
121/121 - 6s - 46ms/step - loss: 0.3241 - mae: 0.6159 - val_loss: 0.1394 - val_mae: 0.3678 - learning_rate: 0.

Walk-Forward Validation: 100%|██████████| 485/485 [00:33<00:00, 14.42it/s]


- Mean Absolute Percentage Error (MAPE): 4.7080%
- Directional Accuracy (DA): 54.4330%

--- Selecting Best GRU Model ---
- Naive Model Benchmark - MAPE: 4.7134%, DA: 44.7699%

- Found 1 candidate model(s) that beat the naive model:
- Model: hyperparameters, MAPE: 4.7080%, DA: 54.4330%

--- Best GRU Model Chosen ---
- MAPE: 4.7080%
- DA: 54.4330%


In [ ]:
hps, metrics, beats_baseline = gru_hps_and_metrics

In [ ]:
hps.values

In [ ]:
plot_residuals_analysis(metrics, 'comp', 14, 'GRU', 'Week')
run_ljung_box_test(metrics, 'comp', 14, 'GRU')
print(metrics['comp'])

Plot saved to 'outputs/plots/gru_residuals_analysis_week'.
= Ljung-Box Test: GRU =
Lags tested: 14
p-value: 0.0000
Result: Reject the null hypothesis (p < 0.05).
The residuals show evidence of autocorrelation.
               true          pred
2022-10-19  19226.0  19158.227504
2022-10-20  19061.0  19393.451301
2022-10-21  19189.0  19187.255122
2022-10-22  19186.0  19123.194173
2022-10-23  19507.0  19393.451301
...             ...           ...
2024-02-11  48295.0  42819.739519
2024-02-12  49745.0  42718.643335
2024-02-13  49126.0  43217.117592
2024-02-14  51475.0  43666.545186
2024-02-15  52003.0  45326.124142

[485 rows x 2 columns]


### GRU Model Tuning (1-Month)

In [ ]:
tf.random.set_seed(42)

In [ ]:
FULL_COLS = list(train_prep_ds.columns)
TARGET_COL = 'close'
INPUT_WINDOW = 30
TARGET_WINDOW = 30
FORECAST_HORIZON = 30

In [ ]:
naive_metrics, naive_pred = train_and_forecast_with_naive_model(valid_prep_ds, 'close', 30, 'Validation', 'Days')

= Naive Model Training, Evaluation, and Forecasting on Validation Set (Horizon: 30 Days) =
--- Fitting Naive Model ---

--- Evaluating Naive Model ---
- Mean Absolute Percentage Error (MAPE): 10.5813%
- Directional Accuracy (DA): 54.8611%

--- Generating Final Naive Forecast (Threshold: 0.5%) ---
- Forecast for 2024-03-16: $52003.00
- Directional Signal for 2024-03-16: 1.0


In [ ]:
gru_hps_and_metrics = find_best_rnn_hps(
    'gru', train_prep_ds, valid_prep_ds,
     FULL_COLS, TARGET_COL, FORECAST_HORIZON,
     INPUT_WINDOW, TARGET_WINDOW,
     'Days', naive_metrics, 15
)

Trial 15 Complete [00h 04m 24s]
val_mae: 0.3572888970375061

Best val_mae So Far: 0.3554573059082031
Total elapsed time: 00h 24m 26s

GRU hyperparameter tuning complete:
- n_conv_layers: 2
- kernel_size: 5
- n_rnn_layers: 3
- use_l2: True
- dropout: 0.30000000000000004
- learning_rate: 0.005117045969850107
- optimizer: adam
- clipnorm: 1.1
- filters_0: 96
- units_0: 96
- filters_1: 192
- units_1: 160
- filters_2: 224
- units_2: 32
- l2_rate: 0.005343634220268847

--- Fitting GRU Model ---
Epoch 1/100
119/119 - 15s - 123ms/step - loss: 0.9964 - mae: 0.7029 - val_loss: 0.1383 - val_mae: 0.3627 - learning_rate: 0.0051
Epoch 2/100
119/119 - 7s - 61ms/step - loss: 0.3264 - mae: 0.6206 - val_loss: 0.1320 - val_mae: 0.3577 - learning_rate: 0.0051
Epoch 3/100
119/119 - 7s - 60ms/step - loss: 0.3267 - mae: 0.6172 - val_loss: 0.1330 - val_mae: 0.3599 - learning_rate: 0.0051
Epoch 4/100
119/119 - 7s - 61ms/step - loss: 0.3240 - mae: 0.6152 - val_loss: 0.1326 - val_mae: 0.3604 - learning_rate: 0.0

Walk-Forward Validation: 100%|██████████| 462/462 [00:33<00:00, 13.61it/s]

- Mean Absolute Percentage Error (MAPE): 10.5712%
- Directional Accuracy (DA): 54.7619%

--- Selecting Best GRU Model ---
- Naive Model Benchmark → MAPE: 10.5813%, DA: 54.8611%

- Conclusion: No GRU model outperformed the naive model.


In [ ]:
hps, metrics, beats_baseline = gru_hps_and_metrics

In [ ]:
hps.values

{'n_conv_layers': 2,
 'kernel_size': 5,
 'n_rnn_layers': 3,
 'use_l2': True,
 'dropout': 0.30000000000000004,
 'learning_rate': 0.005117045969850107,
 'optimizer': 'adam',
 'clipnorm': 1.1,
 'filters_0': 96,
 'units_0': 96,
 'filters_1': 192,
 'units_1': 160,
 'filters_2': 224,
 'units_2': 32,
 'l2_rate': 0.005343634220268847}

In [ ]:
plot_residuals_analysis(metrics, 'comp', 90, 'GRU', 'Month')
run_ljung_box_test(metrics, 'comp', 90, 'GRU')
print(metrics['comp'])

Plot saved to 'outputs/plots/gru_residuals_analysis_month'.
= Ljung-Box Test: GRU =
Lags tested: 90
p-value: 0.0000
Result: Reject the null hypothesis (p < 0.05).
The residuals show evidence of autocorrelation.
               true          pred
2022-11-11  16806.0  19156.895850
2022-11-12  16842.0  19392.103296
2022-11-13  16499.0  19185.921449
2022-11-14  16590.0  19121.864953
2022-11-15  16721.0  19392.103296
...             ...           ...
2024-02-11  48295.0  43462.332569
2024-02-12  49745.0  42831.776436
2024-02-13  49126.0  42919.854118
2024-02-14  51475.0  43016.939745
2024-02-15  52003.0  43042.962697

[462 rows x 2 columns]


### Save hyperparameters to a YAML file.

In [ ]:
hps_to_save = {
    'GRU': {
        'weekly': {
            'n_conv_layers': 2,
            'kernel_size': 5,
            'n_rnn_layers': 3,
            'use_l2': True,
            'dropout': 0.30000000000000004,
            'learning_rate': 0.005117045969850107,
            'optimizer': 'adam',
            'clipnorm': 1.1,
            'filters_0': 96,
            'units_0': 96,
            'filters_1': 192,
            'units_1': 160,
            'filters_2': 224,
            'units_2': 32,
            'l2_rate': 0.005343634220268847
        },
        'monthly': {
            'n_conv_layers': 2,
            'kernel_size': 5,
            'n_rnn_layers': 3,
            'use_l2': True,
            'dropout': 0.30000000000000004,
            'learning_rate': 0.005117045969850107,
            'optimizer': 'adam',
            'clipnorm': 1.1,
            'filters_0': 96,
            'units_0': 96,
            'filters_1': 192,
            'units_1': 160,
            'filters_2': 224,
            'units_2': 32,
            'l2_rate': 0.005343634220268847
        },
    }
}

In [ ]:
save_config(hps_to_save, CONFIG_FILE_PATH)

Configuration successfully saved to '/content/drive/MyDrive/projects/btc/config.yaml'.
